# Converting a Pulser sequence into a Job

In [ ]:
import numpy as np
from qat.core import Variable

Simulation on Pasqal hardware can be performed using [Pulser](https://pulser.readthedocs.io/). This notebook presents how to convert a sequence defined in Pulser into a `Schedule` or a `Job` that can be executed on MyQLM. 

## Defining Pulser's AQPU

In [ ]:
from pulser import Pulse, Sequence, Register
from pulser.waveforms import CustomWaveform
from pulser.devices import MockDevice
from pulser_myqlm import IsingAQPU, Pheaviside, PSchedule
from pulser_myqlm.myqlmtools import are_equivalent_schedules

On Pulser, any simulation starts by defining a `Device` and a `Register`. Any `PulserAQPU` should be defined by these two objects, or by a sequence directly.

`IsingAQPU` describes a specific AQPU on which only pulses applied on a `Rydberg.Global` channel can be run. This generates Ising Hamiltonians defined by the `Register` and the pulses sent (see next section).

Let's start by creating a device having a `Rydberg.Global` channel, and a square register of 2x2 atoms spaced by $4\mu m$. The `MockDevice` defines a device for simulations on which all channels are implemented.



In [ ]:
device = MockDevice
register = Register.square(2, 4, None)
qpu = IsingAQPU(device, register)

In [ ]:
# You can see the register used by the qpu
qpu.register.draw()

In [ ]:
# Check qpu properties
print(
    "Distances (in um):\n", qpu.distances, "\n"
)  # symetric matrix, for each qubit two qubits are at 4um distance and one at 4 * sqrt(2) um
print("Interactions (in rad/us):\n", qpu.c6_interactions)  # symetric matrix, C_6/R^6

## Defining an Ising Hamiltonian on MyQLM

On Pulser we can solve problems of shape 
$$ H = \hbar\sum_i \frac{\Omega(t)}{2}(\cos(\phi) \sigma_i^x - \sin(\phi) \sigma_i^y) - \delta(t)n_i + \frac{1}{2}\sum_{i\neq j}U_{ij}n_i n_j$$
with $\sigma_i^x$ the Pauli $X$ operator applied on qubit $i$ and $n_i = \frac{1-\sigma_i^z}{2}$ with $\sigma_i^z$ the Pauli $Z$ operator applied on qubit $i$

Let's start by defining the parameters:

In [ ]:
t_variable = Variable("t")  # in ns
u_variable = Variable("u")  # parameter
omega_t = t_variable  # in rad/us
delta_t = 1 - t_variable + u_variable  # in rad/us

We can then compute the Ising Hamiltonians associated to pulses defined by $\Omega$, $\delta$, $\phi$ using `IsingAQPU.hamiltonian`.

- Here is an Hamiltonian associated with a pulse of constant amplitude, zero detuning and phase.
It is equivalent to apply a X gate on each qubit, and a ZZ gate whose coefficient is defined from the C6 interactions.

In [ ]:
t1 = 20  # in ns
H1 = qpu.hamiltonian(1, 0, 0)
print(H1)

- If now the phase is equal to $\pi / 2$, we no longer add a X gate on each qubit but a Y gate.

In [ ]:
t2 = 20  # in ns
H2 = qpu.hamiltonian(1, 0, np.pi / 2)
print(H2)

- It is also possible to define time-dependent pulses, varying in amplitude and detuning. See that the coefficient in front the ZZ operators are unchanged.

In [ ]:
t0 = 16  # in ns
H0 = qpu.hamiltonian(omega_t, delta_t, 0)
print(H0)

The Hamiltonian is implemented as a `Schedule` object. You also have to define the duration of the evolution. A `Schedule` can be implemented on the `IsingAQPU` if the coefficient in front of the ZZ operators match the C6 interactions. Therefore, when using hamiltonians defined with `IsingAQPU`, the sum of the drive coefficients should be equal to 1 along the duration of the `Schedule`.

On Pulser, the duration of Pulses are defined as integers in nanoseconds. To avoid edge effect induced by `qat.core.heaviside`, we defined `qat.core.Pheaviside` that is equal to 0 if the argument is equal to the final bound.

`PSchedule` is a sub-class of `Schedule` that performs temporal composition, merging and time shifting of schedules using `Pheaviside` instead of `heaviside`. 

In [ ]:
Pschedule0 = PSchedule(drive=[(1, H0)], tmax=t0)
Pschedule1 = PSchedule(drive=[(1, H1)], tmax=t1)
Pschedule2 = PSchedule(drive=[(1, H2)], tmax=t2)

In [ ]:
Pschedule = Pschedule0 | Pschedule1 | Pschedule2
print(Pschedule)

Here is an equivalent definition of the former `PSchedule`, using `Pheaviside` explicitly instead of temporal compositions.

In [ ]:
Pschedule_sum = PSchedule(
    drive=[
        [Pheaviside(t_variable, 0, t0), H0],
        [Pheaviside(t_variable, t0, t1 + t0), H1],
        [Pheaviside(t_variable, t1 + t0, t1 + t0 + t2), H2],
    ],
    tmax=t1 + t0 + t2,
)
print(Pschedule_sum)

In [ ]:
are_equivalent_schedules(Pschedule, Pschedule_sum)

This `PSchedule` could have also been built from a Pulser `Sequence` by adding the pulses described above.

In [ ]:
seq = Sequence(register, device)
seq.declare_channel("ryd_glob", "rydberg_global")
seq.add(
    Pulse(
        CustomWaveform([omega_t(t=ti) for ti in range(t0)]),
        CustomWaveform(
            [delta_t(t=ti, u=0) for ti in range(t0)]
        ),  # no parametrized sequence for the moment
        0,
    ),
    "ryd_glob",
)
seq.add(Pulse.ConstantPulse(t1, 1, 0, 0), "ryd_glob")
seq.add(Pulse.ConstantPulse(t2, 1, 0, np.pi / 2), "ryd_glob")

In [ ]:
Pschedule_from_seq = IsingAQPU.convert_sequence_to_schedule(seq)

In [ ]:
are_equivalent_schedules(Pschedule_from_seq, Pschedule(u=0))